# **SPAR10: Redaction**

### Installing the required libraries

In [1]:
pip install PyMuPDF==1.16.14

In [2]:
pip install aspose.words

### Importing the required libraries

In [3]:
# NLP libraries
import spacy
from spacy import displacy
nlp = spacy.load('en')

# Time Pkg
import time
timestr = time.strftime("%Y%m%d-%H%M%S")

# for opening pdf
import fitz

# for dealing for docx
import aspose.words as aw

# fr ignoring warnings
import warnings
warnings.filterwarnings('ignore')

Exception ignored in: 'FieldInitializationWarning'
ImportError: the static field 'transparent' of type 'aspose.pydrawing.Brushes' is not initialized
Exception ignored in: 'FieldInitializationWarning'
ImportError: the static field 'alice_blue' of type 'aspose.pydrawing.Brushes' is not initialized
Exception ignored in: 'FieldInitializationWarning'
ImportError: the static field 'antique_white' of type 'aspose.pydrawing.Brushes' is not initialized


# CODE

In [4]:
def get_sensitive_data(lines,tt):
	
  docx = nlp(lines)
  redacted_sentences = []
  for ent in docx.ents:
    ent.merge()
  for token in docx:
    if token.ent_type_ == tt:
      yield token.string

In [5]:
def get_path():

  # replace it with name of the pdf file
  path = "testing.pdf"

  if path[-4:]=='docx':
    
    # Load word document
    doc = aw.Document("testing.docx")
		# Save as PDF
    doc.save("testing.pdf")
    path="testing.pdf"

  return path

In [6]:
def redaction():
	
  """ main redactor code """

  redactables = ['PERSON','GPE','LOC','ORG','TIME','DATE','MONEY','FAC','QUANITY','CARDINAL','ORDINAL']
    
  # get the path of the pdf
  path = get_path()
		
	# opening the pdf
  doc = fitz.open(path)
		
	# iterating through pages
  for page in doc:
		
	  # _wrapContents is needed for fixing
	  # alignment issues with rect boxes in some
		# cases where there is alignment issue
	  page._wrapContents()
	 
	  for i in redactables:
				
			# getting the rect boxes which consists the matching email regex
		  sensitive = get_sensitive_data(page.getText("text"),i)
				
		  for data in sensitive:
			  areas = page.searchFor(data)
					
				# drawing outline over sensitive datas
			  [page.addRedactAnnot(area, fill = (0, 0, 0)) for area in areas]
					
				# applying the redaction
			  page.apply_redactions()
			
  # saving it to a new pdf
  doc.save('redacted.pdf')
  print("Successfully redacted")	

In [7]:
redaction()

Successfully redacted
